###### General Steps to Follow
1. Importing Packages
1. Criteria 1 - Angles
1. Criteria 2 - Angles
1. Criteria 3 - Angles
1. Criteria 4 - Angles
1. Full Test

# -------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf

2024-04-27 21:10:02.832148: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 21:10:03.899586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# -------------------------------------------------------------------------------------------------------------

In [2]:
def evaluate(output, y_true):
    y_pred = np.zeros((output.shape[0],1), dtype = "int")
    y_pred[(output > 0.5)] = 1
    y_pred = y_pred.reshape(y_pred.shape[0])
    print("accuracy =", accuracy_score(y_true, y_pred))

In [3]:
def reshape_input_sequence(data, shape):
    new_data = []
    for training_example in data:
        new_data.append(training_example[0:shape])
    new_data = np.array(new_data)
    return new_data

## 2) Criteria 1 - Angles

#### Loading training data

In [4]:
criteria1_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_1.npy", allow_pickle = True)

In [5]:
len(criteria1_data_angles)

954

#### Getting X_train, X_test, y_train, y_test

In [6]:
X = np.vstack(criteria1_data_angles[:, 0])
y = np.vstack(criteria1_data_angles[:, 1])
X_train_criteria1_angles, X_test_criteria1_angles, y_train_criteria1_angles, y_test_criteria1_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [7]:
print("X_train_criteria1_angles shape:", X_train_criteria1_angles.shape)
print("y_train_criteria1_angles shape:", y_train_criteria1_angles.shape)
print("X_test_criteria1_angles shape:", X_test_criteria1_angles.shape)
print("y_test_criteria1_angles shape:", y_test_criteria1_angles.shape)

X_train_criteria1_angles shape: (763, 600, 8)
y_train_criteria1_angles shape: (763, 1)
X_test_criteria1_angles shape: (191, 600, 8)
y_test_criteria1_angles shape: (191, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [10]:
new_length = 150
X_train_criteria1_angles = reshape_input_sequence(X_train_criteria1_angles, new_length)
X_test_criteria1_angles = reshape_input_sequence(X_test_criteria1_angles, new_length)
print("X_train_criteria1_angles shape:", X_train_criteria1_angles.shape)
print("y_train_criteria1_angles shape:", y_train_criteria1_angles.shape)
print("X_test_criteria1_angles shape:", X_test_criteria1_angles.shape)
print("y_test_criteria1_angles shape:", y_test_criteria1_angles.shape)

X_train_criteria1_angles shape: (763, 150, 8)
y_train_criteria1_angles shape: (763, 1)
X_test_criteria1_angles shape: (191, 150, 8)
y_test_criteria1_angles shape: (191, 1)


#### Building and training the model

In [84]:
criteria1_angles_model = Sequential(
    [
        GRU(units=265, input_shape=(X_train_criteria1_angles.shape[1], X_train_criteria1_angles.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], 
    name="criteria1_angles_model"
)

criteria1_angles_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria1_angles_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ (None, 150, 265)       │       218,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 150, 265)       │         1,060 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 150, 128)       │       151,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 150, 64)        │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 418,950 (1.60 MB)

 Trainable params: 417,972 (1.59 MB)

 Non-trainable params: 978 (3.82 KB)

In [92]:
criteria1_angles_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [93]:
criteria1_angles_model.fit(X_train_criteria1_angles,y_train_criteria1_angles, epochs = 5, batch_size = 32)

Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 343ms/step - accuracy: 0.9930 - loss: 0.0295
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 330ms/step - accuracy: 0.9967 - loss: 0.0202
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 316ms/step - accuracy: 0.9964 - loss: 0.0139
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 328ms/step - accuracy: 0.9993 - loss: 0.0098
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 331ms/step - accuracy: 0.9948 - loss: 0.0225


#### Saving the model

In [100]:
criteria1_angles_model.save("../models/other/bicep/angles/bicep_criteria1_angles_model_v2.h5")

#### Loading the model

In [11]:
criteria1_angles_model = tf.keras.models.load_model('../models/other/bicep/angles/bicep_criteria1_angles_model_v2.h5')

2024-04-27 21:10:35.088726: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-27 21:10:35.094037: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


#### Evaluating the model

###### Train Data

In [12]:
y_pred = criteria1_angles_model.predict(X_train_criteria1_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria1_angles)

24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step
accuracy = 0.963302752293578


###### Test Data

In [13]:
y_pred = criteria1_angles_model.predict(X_test_criteria1_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria1_angles)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step
accuracy = 0.9685863874345549


# -------------------------------------------------------------------------------------------------------------

## 3) Criteria 2 - Angles

#### Loading training data

In [14]:
criteria2_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_2.npy", allow_pickle = True)

In [15]:
len(criteria2_data_angles)

1064

#### Getting X_train, X_test, y_train, y_test

In [16]:
X = np.vstack(criteria2_data_angles[:, 0])
y = np.vstack(criteria2_data_angles[:, 1])
X_train_criteria2_angles, X_test_criteria2_angles, y_train_criteria2_angles, y_test_criteria2_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [17]:
print("X_train_criteria2_angles shape:", X_train_criteria2_angles.shape)
print("y_train_criteria2_angles shape:", y_train_criteria2_angles.shape)
print("X_test_criteria2_angles shape:", X_test_criteria2_angles.shape)
print("y_test_criteria2_angles shape:", y_test_criteria2_angles.shape)

X_train_criteria2_angles shape: (851, 600, 8)
y_train_criteria2_angles shape: (851, 1)
X_test_criteria2_angles shape: (213, 600, 8)
y_test_criteria2_angles shape: (213, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [18]:
new_length = 150
X_train_criteria2_angles = reshape_input_sequence(X_train_criteria2_angles, new_length)
X_test_criteria2_angles = reshape_input_sequence(X_test_criteria2_angles, new_length)
print("X_train_criteria2_angles shape:", X_train_criteria2_angles.shape)
print("y_train_criteria2_angles shape:", y_train_criteria2_angles.shape)
print("X_test_criteria2_angles shape:", X_test_criteria2_angles.shape)
print("y_test_criteria2_angles shape:", y_test_criteria2_angles.shape)

X_train_criteria2_angles shape: (851, 150, 8)
y_train_criteria2_angles shape: (851, 1)
X_test_criteria2_angles shape: (213, 150, 8)
y_test_criteria2_angles shape: (213, 1)


#### Building and training the model

In [120]:
criteria2_angles_model = Sequential(
    [
        GRU(units=200, input_shape=(X_train_criteria2_angles.shape[1], X_train_criteria2_angles.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria2_angles_model"
)
criteria2_angles_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria2_angles_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_12 (GRU)                    │ (None, 150, 200)       │       126,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 150, 200)       │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_13 (GRU)                    │ (None, 150, 128)       │       126,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_14 (GRU)                    │ (None, 150, 128)       │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_15 (GRU)                    │ (None, 32)             │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 369,329 (1.41 MB)

 Trainable params: 368,353 (1.41 MB)

 Non-trainable params: 976 (3.81 KB)

In [125]:
criteria2_angles_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [129]:
criteria2_angles_model.fit(X_train_criteria2_angles,y_train_criteria2_angles, epochs = 5, batch_size = 32)

Epoch 1/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 384ms/step - accuracy: 0.9829 - loss: 0.0323
Epoch 2/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 374ms/step - accuracy: 0.9989 - loss: 0.0132
Epoch 3/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 366ms/step - accuracy: 0.9992 - loss: 0.0107
Epoch 4/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 362ms/step - accuracy: 0.9998 - loss: 0.0078
Epoch 5/5
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 352ms/step - accuracy: 0.9951 - loss: 0.0134


#### Saving the model

In [133]:
criteria2_angles_model.save("../models/other/bicep/angles/bicep_criteria2_angles_model_v2.h5")

#### Loading the model

In [19]:
criteria2_angles_model = tf.keras.models.load_model('../models/other/bicep/angles/bicep_criteria2_angles_model_v2.h5')

#### Evaluating the model

###### Train Data

In [20]:
y_pred = criteria2_angles_model.predict(X_train_criteria2_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria2_angles)

27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step
accuracy = 0.9694477085781433


###### Test Data

In [21]:
y_pred = criteria2_angles_model.predict(X_test_criteria2_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria2_angles)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step
accuracy = 0.9671361502347418


# -------------------------------------------------------------------------------------------------------------

## 4) Criteria 3 - Angles

#### Loading training data

In [22]:
criteria3_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_3.npy", allow_pickle = True)

In [23]:
len(criteria3_data_angles)

767

#### Getting X_train, X_test, y_train, y_test

In [24]:
X = np.vstack(criteria3_data_angles[:, 0])
y = np.vstack(criteria3_data_angles[:, 1])
X_train_criteria3_angles, X_test_criteria3_angles, y_train_criteria3_angles, y_test_criteria3_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [25]:
print("X_train_criteria3_angles shape:", X_train_criteria3_angles.shape)
print("y_train_criteria3_angles shape:", y_train_criteria3_angles.shape)
print("X_test_criteria3_angles shape:", X_test_criteria3_angles.shape)
print("y_test_criteria3_angles shape:", y_test_criteria3_angles.shape)

X_train_criteria3_angles shape: (613, 600, 8)
y_train_criteria3_angles shape: (613, 1)
X_test_criteria3_angles shape: (154, 600, 8)
y_test_criteria3_angles shape: (154, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [26]:
new_length = 150
X_train_criteria3_angles = reshape_input_sequence(X_train_criteria3_angles, new_length)
X_test_criteria3_angles = reshape_input_sequence(X_test_criteria3_angles, new_length)
print("X_train_criteria3_angles shape:", X_train_criteria3_angles.shape)
print("y_train_criteria3_angles shape:", y_train_criteria3_angles.shape)
print("X_test_criteria3_angles shape:", X_test_criteria3_angles.shape)
print("y_test_criteria3_angles shape:", y_test_criteria3_angles.shape)

X_train_criteria3_angles shape: (613, 150, 8)
y_train_criteria3_angles shape: (613, 1)
X_test_criteria3_angles shape: (154, 150, 8)
y_test_criteria3_angles shape: (154, 1)


#### Building and training the model

In [145]:
criteria3_angles_model = Sequential(
    [
        GRU(units=256, input_shape=(X_train_criteria3_angles.shape[1], X_train_criteria3_angles.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 150, return_sequences=True),
        BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 20),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria3_angles_model"
)
criteria3_angles_model.summary()

Model: "criteria3_angles_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_20 (GRU)                    │ (None, 150, 256)       │       204,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 150, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_21 (GRU)                    │ (None, 150, 150)       │       183,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 150, 150)       │           600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_22 (GRU)                    │ (None, 150, 64)        │        41,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_23 (GRU)                    │ (None, 20)             │         5,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 20)             │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 436,501 (1.67 MB)

 Trainable params: 435,521 (1.66 MB)

 Non-trainable params: 980 (3.83 KB)

In [150]:
criteria3_angles_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [151]:
criteria3_angles_model.fit(X_train_criteria3_angles, y_train_criteria3_angles, epochs = 5, batch_size = 32)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 374ms/step - accuracy: 0.9306 - loss: 0.1889
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 360ms/step - accuracy: 0.9593 - loss: 0.1156
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 368ms/step - accuracy: 0.9749 - loss: 0.0821
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 367ms/step - accuracy: 0.9726 - loss: 0.0832
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 370ms/step - accuracy: 0.9871 - loss: 0.0588


#### Saving the model

In [154]:
criteria3_angles_model.save("../models/other/bicep/angles/bicep_criteria3_angles_model_v3.h5")

#### Loading the model

In [27]:
criteria3_angles_model = tf.keras.models.load_model('../models/other/bicep/angles/bicep_criteria3_angles_model_v3.h5')

#### Evaluating the model

###### Train Data

In [28]:
y_pred = criteria3_angles_model.predict(X_train_criteria3_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria3_angles)

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step
accuracy = 0.965742251223491


###### Test Data

In [29]:
y_pred = criteria3_angles_model.predict(X_test_criteria3_angles)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria3_angles)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step
accuracy = 0.948051948051948


# -------------------------------------------------------------------------------------------------------------

## 5) Criteria 4 - Angles

#### Loading training data

In [32]:
criteria4_data_angles = np.load("../data/processed/bicep/angles/training_data_bicep_angles_criteria_4.npy", allow_pickle = True)

In [33]:
len(criteria4_data_angles)

803

#### Getting X_train, X_test, y_train, y_test

In [34]:
X = np.vstack(criteria4_data_angles[:, 0])
y = np.vstack(criteria4_data_angles[:, 1])
X_train_criteria4_angles, X_test_criteria4_angles, y_train_criteria4_angles, y_test_criteria4_angles = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [35]:
print("X_train_criteria4_angles shape:", X_train_criteria4_angles.shape)
print("y_train_criteria4_angles shape:", y_train_criteria4_angles.shape)
print("X_test_criteria4_angles shape:", X_test_criteria4_angles.shape)
print("y_test_criteria4_angles shape:", y_test_criteria4_angles.shape)

X_train_criteria4_angles shape: (642, 600, 8)
y_train_criteria4_angles shape: (642, 1)
X_test_criteria4_angles shape: (161, 600, 8)
y_test_criteria4_angles shape: (161, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [36]:
new_length = 150
X_train_criteria4_angles = reshape_input_sequence(X_train_criteria4_angles, new_length)
X_test_criteria4_angles = reshape_input_sequence(X_test_criteria4_angles, new_length)
print("X_train_criteria4_angles shape:", X_train_criteria4_angles.shape)
print("y_train_criteria4_angles shape:", y_train_criteria4_angles.shape)
print("X_test_criteria4_angles shape:", X_test_criteria4_angles.shape)
print("y_test_criteria4_angles shape:", y_test_criteria4_angles.shape)

X_train_criteria4_angles shape: (642, 150, 8)
y_train_criteria4_angles shape: (642, 1)
X_test_criteria4_angles shape: (161, 150, 8)
y_test_criteria4_angles shape: (161, 1)


#### Building and training the model

In [38]:
criteria4_angles_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria4_angles.shape[1], X_train_criteria4_angles.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria4_angles_model"
)
criteria4_angles_model.summary()

Model: "criteria4_angles_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 50)                11800     
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11851 (46.29 KB)
Trainable params: 11851 (46.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
criteria4_angles_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [40]:
criteria4_angles_model.fit(X_train_criteria4_angles, y_train_criteria4_angles, epochs = 10)

Epoch 1/10
21/21 [==============================] - 5s 191ms/step - loss: 0.6852 - accuracy: 0.5857
Epoch 2/10
21/21 [==============================] - 3s 168ms/step - loss: 0.6678 - accuracy: 0.6199
Epoch 3/10
21/21 [==============================] - 4s 180ms/step - loss: 0.6645 - accuracy: 0.6199
Epoch 4/10
21/21 [==============================] - 4s 195ms/step - loss: 0.6654 - accuracy: 0.6199
Epoch 5/10
21/21 [==============================] - 4s 210ms/step - loss: 0.6643 - accuracy: 0.6199
Epoch 6/10
21/21 [==============================] - 4s 203ms/step - loss: 0.6659 - accuracy: 0.6199
Epoch 7/10
21/21 [==============================] - 4s 190ms/step - loss: 0.6653 - accuracy: 0.6199
Epoch 8/10
21/21 [==============================] - 4s 202ms/step - loss: 0.6660 - accuracy: 0.6199
Epoch 9/10
21/21 [==============================] - 4s 190ms/step - loss: 0.6651 - accuracy: 0.6199
Epoch 10/10
21/21 [==============================] - 4s 186ms/step - loss: 0.6658 - accuracy: 0.6199

#### Saving the model

In [ ]:
criteria4_angles_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [41]:
y_pred = criteria4_angles_model.predict(X_train_criteria4_angles)
evaluate(y_pred, y_train_criteria4_angles)

21/21 [==============================] - 2s 71ms/step
accuracy = 0.6199376947040498


###### Test Data

In [42]:
y_pred = criteria4_angles_model.predict(X_test_criteria4_angles)
evaluate(y_pred, y_test_criteria4_angles)

6/6 [==============================] - 0s 62ms/step
accuracy = 0.6024844720496895


# -------------------------------------------------------------------------------------------------------------

## 6) Full Test

In [37]:
def test_models(input_sample):
    input_sample = input_sample.reshape((1, input_sample.shape[0], input_sample.shape[1]))
    y_pred1 = criteria1_angles_model.predict(input_sample)
    y_pred1 = tf.nn.sigmoid(y_pred1)
    y_pred2 = criteria2_angles_model.predict(input_sample)
    y_pred2 = tf.nn.sigmoid(y_pred2)
    y_pred3 = criteria3_angles_model.predict(input_sample)
    y_pred3 = tf.nn.sigmoid(y_pred3)
    print("Criteria1:", y_pred1[0][0].numpy())
    print("Criteria2:", y_pred2[0][0].numpy())
    print("Criteria3:", y_pred3[0][0].numpy())

In [39]:
import random

In [57]:
num = random.randint(0,X_test_criteria4_angles.shape[0]-1)
input_sample = X_test_criteria4_angles[num]
print(y_test_criteria4_angles[num])
test_models(input_sample)
# results are good for criteria1_v2, criteria2_v1, criteria3_v2

[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Criteria1: 0.9959381
Criteria2: 0.36555153
Criteria3: 0.43953305


In [53]:
input_sample = X_train_criteria2_angles[3]
print(y_train_criteria2_angles[3])
test_models(input_sample)

[1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Criteria1: 0.026292637
Criteria2: 0.99960154
Criteria3: 0.17431119


In [55]:
input_sample = input_sample.reshape((1, input_sample.shape[0], input_sample.shape[1]))
y_pred1 = criteria1_angles_model.predict(input_sample)
y_pred1 = tf.nn.sigmoid(y_pred1)
y_pred1 = y_pred1[0][0].numpy()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
